# Interactive Parking Availability Analysis

**Data Source:** South Tyrol Open Data Hub API  
**Coverage:** Val Gardena, Brunico, Bressanone (20 parking stations)  
**Data Points:** 2M+ records across 59 weeks  

Use the widgets below to explore parking patterns interactively.


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print('OK: Libraries loaded')

In [ ]:
# Load data with robust error handling
print('Loading data...')
csv_path = 'data/parking_data_dolomites.csv'

df = pd.read_csv(csv_path, comment='#', encoding='utf-8')
print(f'Loaded {len(df):,} raw records')

# Parse timestamps
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date

# Convert to numeric - keep as float to avoid casting errors
df['available'] = pd.to_numeric(df['available'], errors='coerce')
df['capacity'] = pd.to_numeric(df['capacity'], errors='coerce')

# Remove rows with no available data
df = df.dropna(subset=['available'])

# Fill missing capacity with 0
df['capacity'] = df['capacity'].fillna(0)

# Extract time components
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['time_of_day'] = df['hour'] * 60 + df['minute']
df['day_of_week'] = df['timestamp'].dt.day_name()
df['month'] = df['timestamp'].dt.strftime('%Y-%m')
df['time_bucket'] = (df['time_of_day'] // 5) * 5

# Filter to 7am-6pm (420 to 1080 minutes)
df_filtered = df[(df['time_of_day'] >= 420) & (df['time_of_day'] <= 1080)].copy()

# Calculate occupancy % safely
df_filtered['occupancy_pct'] = 0.0
mask = df_filtered['capacity'] > 0
df_filtered.loc[mask, 'occupancy_pct'] = (df_filtered.loc[mask, 'available'] / df_filtered.loc[mask, 'capacity'] * 100)

print(f'OK: Filtered to {len(df_filtered):,} records (7am-6pm)')
print(f'OK: {df_filtered["name"].nunique()} locations, {df_filtered["month"].nunique()} months')
print(f'OK: Date range: {df_filtered["date"].min()} to {df_filtered["date"].max()}')

In [ ]:
# Pre-compute aggregations
print('Pre-computing aggregations...')

# Hourly averages
hourly_agg = df_filtered.groupby(['region', 'name', 'month', 'hour']).agg(
    available=('available', 'mean'),
    occupancy_pct=('occupancy_pct', 'mean')
).reset_index()

# Weekly heatmap (hour x day)
weekly_agg = df_filtered.groupby(['region', 'name', 'month', 'hour', 'day_of_week']).agg(
    available=('available', 'mean'),
    occupancy_pct=('occupancy_pct', 'mean')
).reset_index()

print('OK: Aggregations complete')

In [ ]:
# Helper functions

def get_regions():
    return ['All'] + sorted(df_filtered['region'].unique().tolist())

def get_locations(region='All'):
    if region == 'All':
        return sorted(df_filtered['name'].unique().tolist())
    return sorted(df_filtered[df_filtered['region'] == region]['name'].unique().tolist())

def get_months():
    return sorted(df_filtered['month'].unique().tolist())

def get_color_map(items):
    cmap = px.colors.sequential.Turbo
    items_sorted = sorted(items)
    n = len(items_sorted)
    return {item: cmap[int(i * (len(cmap)-1) / max(1, n-1))] for i, item in enumerate(items_sorted)}

print('OK: Helper functions defined')

## Summary Statistics

In [ ]:
print('\n' + '='*70)
print('SUMMARY STATISTICS')
print('='*70)
print(f'Total records: {len(df_filtered):,}')
print(f'Locations: {df_filtered["name"].nunique()}')
print(f'Months: {df_filtered["month"].nunique()}')
print(f'Date range: {df_filtered["date"].min()} to {df_filtered["date"].max()}')
print(f'\nAverage available: {df_filtered["available"].mean():.1f} spaces')
print(f'Average occupancy: {df_filtered["occupancy_pct"].mean():.1f}%')
print('='*70)

## Interactive Line Plot

In [ ]:
# Widgets
region_dd = widgets.Dropdown(options=get_regions(), value='Val Gardena', description='Region:')
month_select = widgets.SelectMultiple(options=get_months(), value=[get_months()[-1]], description='Months:', rows=5)
location_select = widgets.SelectMultiple(options=get_locations('Val Gardena'), description='Locations:', rows=5)
metric_dd = widgets.Dropdown(options=['Available Spaces', 'Occupancy %'], value='Available Spaces', description='Metric:')

# Update locations when region changes
def on_region_change(change):
    location_select.options = get_locations(region_dd.value)

region_dd.observe(on_region_change, 'value')

output = widgets.Output()

def plot_line(region, months, locations, metric):
    if not months or not locations:
        return
    
    with output:
        output.clear_output(wait=True)
        
        # Filter data
        data = hourly_agg[
            (hourly_agg['month'].isin(months)) &
            (hourly_agg['name'].isin(locations))
        ]
        
        if region != 'All':
            data = data[data['region'] == region]
        
        if data.empty:
            print('No data for selection')
            return
        
        fig = go.Figure()
        color_map = get_color_map(locations)
        
        for location in sorted(locations):
            loc_data = data[data['name'] == location].sort_values('hour')
            
            y_col = 'occupancy_pct' if metric == 'Occupancy %' else 'available'
            
            fig.add_trace(go.Scatter(
                x=[f'{h:02d}:00' for h in loc_data['hour']],
                y=loc_data[y_col],
                name=location,
                mode='lines+markers',
                line=dict(color=color_map[location], width=2),
                marker=dict(size=6)
            ))
        
        fig.update_layout(
            title=f'{metric} by Hour - {region}',
            xaxis_title='Hour of Day',
            yaxis_title=metric,
            height=500,
            hovermode='x unified'
        )
        fig.show()

display(widgets.HBox([
    widgets.VBox([region_dd, month_select, location_select, metric_dd]),
    output
]))

# Initial plot
plot_line(region_dd.value, month_select.value, location_select.value or [get_locations('Val Gardena')[0]], metric_dd.value)

# Observe changes
widgets.interactive_output(plot_line, {
    'region': region_dd,
    'months': month_select,
    'locations': location_select,
    'metric': metric_dd
})

## Interactive Heatmap - Weekly Patterns

In [ ]:
hm_region_dd = widgets.Dropdown(options=get_regions(), value='Val Gardena', description='Region:')
hm_location_dd = widgets.Dropdown(options=get_locations('Val Gardena'), description='Location:')
hm_month_dd = widgets.Dropdown(options=get_months(), value=get_months()[-1], description='Month:')
hm_metric_dd = widgets.Dropdown(options=['Available Spaces', 'Occupancy %'], value='Available Spaces', description='Metric:')

def on_hm_region_change(change):
    locs = get_locations(hm_region_dd.value)
    hm_location_dd.options = locs
    hm_location_dd.value = locs[0] if locs else None

hm_region_dd.observe(on_hm_region_change, 'value')

hm_output = widgets.Output()

def plot_heatmap(region, location, month, metric):
    with hm_output:
        hm_output.clear_output(wait=True)
        
        data = weekly_agg[
            (weekly_agg['month'] == month) &
            (weekly_agg['name'] == location)
        ]
        
        if region != 'All':
            data = data[data['region'] == region]
        
        if data.empty:
            print('No data for selection')
            return
        
        y_col = 'occupancy_pct' if metric == 'Occupancy %' else 'available'
        
        pivot = data.pivot_table(index='hour', columns='day_of_week', values=y_col, aggfunc='mean')
        
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        pivot = pivot[[d for d in day_order if d in pivot.columns]]
        
        fig = go.Figure(data=go.Heatmap(
            z=pivot.values,
            x=pivot.columns,
            y=[f'{h:02d}:00' for h in pivot.index],
            colorscale='RdYlGn' if metric == 'Available Spaces' else 'RdYlGn_r',
            colorbar=dict(title=metric)
        ))
        
        fig.update_layout(
            title=f'{location} - {month} Weekly Pattern',
            xaxis_title='Day of Week',
            yaxis_title='Hour of Day',
            height=600
        )
        fig.show()

display(widgets.HBox([
    widgets.VBox([hm_region_dd, hm_location_dd, hm_month_dd, hm_metric_dd]),
    hm_output
]))

plot_heatmap(hm_region_dd.value, hm_location_dd.value, hm_month_dd.value, hm_metric_dd.value)

widgets.interactive_output(plot_heatmap, {
    'region': hm_region_dd,
    'location': hm_location_dd,
    'month': hm_month_dd,
    'metric': hm_metric_dd
})

## Month Comparison

In [ ]:
comp_region_dd = widgets.Dropdown(options=get_regions(), value='Val Gardena', description='Region:')
comp_month_select = widgets.SelectMultiple(options=get_months(), value=get_months()[-2:], description='Months (2-4):', rows=6)
comp_location_select = widgets.SelectMultiple(options=get_locations('Val Gardena'), value=get_locations('Val Gardena')[:2], description='Locations:', rows=4)

def on_comp_region_change(change):
    locs = get_locations(comp_region_dd.value)
    comp_location_select.options = locs
    comp_location_select.value = tuple(locs[:2]) if len(locs) >= 2 else tuple(locs)

comp_region_dd.observe(on_comp_region_change, 'value')

comp_output = widgets.Output()

def plot_comparison(region, months, locations):
    if len(months) < 2 or not locations:
        return
    
    with comp_output:
        comp_output.clear_output(wait=True)
        
        n_months = len(months)
        rows = (n_months + 1) // 2
        
        fig = make_subplots(rows=rows, cols=2, subplot_titles=tuple(sorted(months)))
        
        color_map = get_color_map(locations)
        
        for idx, month in enumerate(sorted(months)):
            row = idx // 2 + 1
            col = idx % 2 + 1
            
            data = hourly_agg[
                (hourly_agg['month'] == month) &
                (hourly_agg['name'].isin(locations))
            ]
            
            if region != 'All':
                data = data[data['region'] == region]
            
            for location in sorted(locations):
                loc_data = data[data['name'] == location].sort_values('hour')
                if not loc_data.empty:
                    fig.add_trace(
                        go.Scatter(
                            x=[f'{h:02d}:00' for h in loc_data['hour']],
                            y=loc_data['available'],
                            name=location,
                            mode='lines+markers',
                            line=dict(color=color_map[location]),
                            showlegend=(idx == 0)
                        ),
                        row=row,
                        col=col
                    )
        
        fig.update_layout(height=400*rows, title_text=f'Month Comparison - {region}', hovermode='x unified')
        fig.show()

display(widgets.HBox([
    widgets.VBox([comp_region_dd, comp_month_select, comp_location_select]),
    comp_output
]))

plot_comparison(comp_region_dd.value, comp_month_select.value, comp_location_select.value)

widgets.interactive_output(plot_comparison, {
    'region': comp_region_dd,
    'months': comp_month_select,
    'locations': comp_location_select
})

## Notes

- Click the camera icon to save any plot
- Hover over charts for detailed values
- Use File → Export as HTML to save the entire notebook
- To refresh with new data: `python scraper_dolomites.py --once` then restart this notebook
